<a href="https://colab.research.google.com/github/martush/martush_notebooks/blob/develop/Char_Level_Language_Model_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
words = open('gdrive/My Drive/names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [5]:
len(words)

32033

In [6]:
#build vocabulary of all chars and mapping to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [8]:
#build dataset
block_size = 3 #context length - how many input chars to predict next
X, Y = [], []

for w in words[:5]:
  print(w)
  context = [0] * block_size
  print(f'Context: {context}')
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    print(''.join(itos[i] for i in context), '---->', itos[ix])
    context = context[1:] + [ix] #crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
Context: [0, 0, 0]
... ----> e
..e ----> m
.em ----> m
emm ----> a
mma ----> .
olivia
Context: [0, 0, 0]
... ----> o
..o ----> l
.ol ----> i
oli ----> v
liv ----> i
ivi ----> a
via ----> .
ava
Context: [0, 0, 0]
... ----> a
..a ----> v
.av ----> a
ava ----> .
isabella
Context: [0, 0, 0]
... ----> i
..i ----> s
.is ----> a
isa ----> b
sab ----> e
abe ----> l
bel ----> l
ell ----> a
lla ----> .
sophia
Context: [0, 0, 0]
... ----> s
..s ----> o
.so ----> p
sop ----> h
oph ----> i
phi ----> a
hia ----> .


In [10]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [13]:
X

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13],
        [13, 13,  1],
        [ 0,  0,  0],
        [ 0,  0, 15],
        [ 0, 15, 12],
        [15, 12,  9],
        [12,  9, 22],
        [ 9, 22,  9],
        [22,  9,  1],
        [ 0,  0,  0],
        [ 0,  0,  1],
        [ 0,  1, 22],
        [ 1, 22,  1],
        [ 0,  0,  0],
        [ 0,  0,  9],
        [ 0,  9, 19],
        [ 9, 19,  1],
        [19,  1,  2],
        [ 1,  2,  5],
        [ 2,  5, 12],
        [ 5, 12, 12],
        [12, 12,  1],
        [ 0,  0,  0],
        [ 0,  0, 19],
        [ 0, 19, 15],
        [19, 15, 16],
        [15, 16,  8],
        [16,  8,  9],
        [ 8,  9,  1]])

Now to create the lookup table. In the reference paper they have 17,000 words crammed into 30 dimensions. We only have 27 characters and we can start  with only 2 dimensions.

In [14]:
C = torch.randn((27,2))

In [15]:
# Lets check a single embedding
C[5]

tensor([-0.0899,  2.5120])

In [20]:
C[torch.tensor([5,6,7,7,7])]

tensor([[-0.0899,  2.5120],
        [ 0.3969,  0.3909],
        [ 1.0112,  1.2152],
        [ 1.0112,  1.2152],
        [ 1.0112,  1.2152]])

In [21]:
# We can also index with multiple dimension
C[X]

tensor([[[-1.5427,  0.5617],
         [-1.5427,  0.5617],
         [-1.5427,  0.5617]],

        [[-1.5427,  0.5617],
         [-1.5427,  0.5617],
         [-0.0899,  2.5120]],

        [[-1.5427,  0.5617],
         [-0.0899,  2.5120],
         [-0.7480, -0.5429]],

        [[-0.0899,  2.5120],
         [-0.7480, -0.5429],
         [-0.7480, -0.5429]],

        [[-0.7480, -0.5429],
         [-0.7480, -0.5429],
         [ 0.0502,  0.1692]],

        [[-1.5427,  0.5617],
         [-1.5427,  0.5617],
         [-1.5427,  0.5617]],

        [[-1.5427,  0.5617],
         [-1.5427,  0.5617],
         [-0.0382,  1.5695]],

        [[-1.5427,  0.5617],
         [-0.0382,  1.5695],
         [ 1.7933,  1.0435]],

        [[-0.0382,  1.5695],
         [ 1.7933,  1.0435],
         [ 1.4301, -0.9535]],

        [[ 1.7933,  1.0435],
         [ 1.4301, -0.9535],
         [-0.0168,  0.4928]],

        [[ 1.4301, -0.9535],
         [-0.0168,  0.4928],
         [ 1.4301, -0.9535]],

        [[-0.0168,  0

In [22]:
C[X].shape

torch.Size([32, 3, 2])

In [23]:
X[13,2]

tensor(1)

In [24]:
C[X][13,2]

tensor([0.0502, 0.1692])

In [25]:
C[1]

tensor([0.0502, 0.1692])

In [16]:
encoded = F.one_hot(torch.tensor(5), num_classes=27)
encoded

tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0])

In [ ]:
# one-hot are int64 and C is float - pytorch doesn't know how to multiply them
encoded @ C

In [ ]:
# identical output
encoded.float() @ C

2 approaches with identical results. Will simply index (C[5]) since much faster

In [ ]:
# Pytorch indexing is quite flexible and powerful
# We have a X with size [32, 3] which we want to embed
# We can index 3 things at the same time
C[[5,6,7]]

In [26]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

Construct the hidden layer

In [ ]:
# torch.Size([32, 3, 2])
# Inputs: 3 x 2 - 3 two-dimensional embeddings
# Outputs we pick - pick 100 neurons
W1 = torch.randn((6, 100))
# biases - initialized randomly, need 100 of them
b1 = torch.randn(100)

In [ ]:
# Normally we'd take the input and multiply it by the weights
emb @ W1 + b1
# Problem is the embeddings are stacked up in the dimensions in the input tensor - 32 x 3 x 1 can't multiply by 6 x 100
